In [1]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
import folium
from folium.plugins import HeatMap
import webbrowser

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
ee.Initialize()

df= pd.read_csv('extracted_tornado.csv')

gdf= gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.BEGIN_LON, df.BEGIN_LAT))

In [5]:
gdf= gpd.read_file('tornado_loc.shp')

In [4]:
gdf['year']= gdf.BEGIN_YEAR//100

In [5]:
gdf['BEGIN_LC']= None
gdf['BEGIN_LU']= None

In [19]:
gdf= gdf[~pd.isna(gdf.BEGIN_LON)]

In [7]:
ee_values= {'landcover':[], 'landuse':[],'coords':[]}
for year in gdf.year.unique():
    if year<=1985:
        tar_year= 1985
    else: tar_year=year
    
    landcover= ee.Image('USFS/GTAC/LCMS/v2020-5/LCMS_CONUS_v2020-5_%d'%tar_year).select(1)
    landuse= ee.Image('USFS/GTAC/LCMS/v2020-5/LCMS_CONUS_v2020-5_%d'%tar_year).select(2)        
    _gdf= gdf[gdf.year==year]
    points= ee.Geometry.MultiPoint([[_gdf.iloc[j].BEGIN_LON, _gdf.iloc[j].BEGIN_LAT] for j in range(len(_gdf))])
    return_js= landcover.sampleRegions(collection=points, scale=30, geometries=True)
    return_dict= return_js.getInfo()
    lcs= [feature['properties']['Land_Cover'] for feature in return_dict['features']]
    return_js= landuse.sampleRegions(collection=points, scale=30, geometries=True)
    return_dict= return_js.getInfo()    
    lus= [feature['properties']['Land_Use'] for feature in return_dict['features']]
    coords= [feature['geometry']['coordinates'] for feature in return_dict['features']]
    ee_values['landcover'].append(lcs)
    ee_values['landuse'].append(lus)
    ee_values['coords'].append(coords)
    
coords= sum(ee_values['coords'],[])
lus= sum(ee_values['landuse'],[])
lcs=sum(ee_values['landcover'],[])

# for i, [lon, lat] in enumerate(coords):
#     inds= gdf[(abs(gdf.BEGIN_LON-lon)<1e-4) & (abs(gdf.BEGIN_LAT-lat)<1e-4)].index
#     gdf.loc[inds,'LandCover']= lcs[i]    
#     gdf.loc[inds,'LandUse']= lus[i]

In [8]:
for i, [lon, lat] in enumerate(coords):
    inds= gdf[(abs(gdf.BEGIN_LON-lon)<1e-3) & (abs(gdf.BEGIN_LAT-lat)<1e-3)].index
    gdf.loc[inds,'LandCover_Begin']= lcs[i]    
    gdf.loc[inds,'LandUse_Begin']= lus[i]

In [9]:
gdf['LandCover_End']= None
gdf['LandUse_End']= None

In [10]:
ee_values= {'landcover':[], 'landuse':[],'coords':[]}
for year in gdf.year.unique():
    if year<=1985:
        tar_year= 1985
    else: tar_year=year
    
    landcover= ee.Image('USFS/GTAC/LCMS/v2020-5/LCMS_CONUS_v2020-5_%d'%tar_year).select(1)
    landuse= ee.Image('USFS/GTAC/LCMS/v2020-5/LCMS_CONUS_v2020-5_%d'%tar_year).select(2)        
    _gdf= gdf[gdf.year==year]
    points= ee.Geometry.MultiPoint([[_gdf.iloc[j].END_LON, _gdf.iloc[j].END_LAT] for j in range(len(_gdf)) if not pd.isna(_gdf.iloc[j].END_LON)])
    return_js= landcover.sampleRegions(collection=points, scale=30, geometries=True)
    return_dict= return_js.getInfo()
    lcs= [feature['properties']['Land_Cover'] for feature in return_dict['features']]
    return_js= landuse.sampleRegions(collection=points, scale=30, geometries=True)
    return_dict= return_js.getInfo()    
    lus= [feature['properties']['Land_Use'] for feature in return_dict['features']]
    coords= [feature['geometry']['coordinates'] for feature in return_dict['features']]
    ee_values['landcover'].append(lcs)
    ee_values['landuse'].append(lus)
    ee_values['coords'].append(coords)
    
coords= sum(ee_values['coords'],[])
lus= sum(ee_values['landuse'],[])
lcs=sum(ee_values['landcover'],[])

In [11]:
for i, [lon, lat] in enumerate(coords):
    inds= gdf[(abs(gdf.END_LON-lon)<1e-3) & (abs(gdf.END_LAT-lat)<1e-3)].index
    gdf.loc[inds,'LandCover_End']= lcs[i]    
    gdf.loc[inds,'LandUse_End']= lus[i]

In [12]:
gdf

,Unnamed_ 0,BEGIN_YEAR,BEGIN_DAY,BEGIN_TIME,END_YEARMO,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,...,DATA_SOURC,year_1,LandCover,LandUse,geometry,year,LandCover_Begin,LandUse_Begin,LandCover_End,LandUse_End
0,0,195004,28,1445,195004,28,1445,NaN,10096222,OKLAHOMA,...,PUB,1950,10,1,POINT (-99.20000 35.12000),1950,15,1,15,1
1,1,195004,29,1530,195004,29,1530,NaN,10120412,TEXAS,...,PUB,1950,12,7,POINT (-98.60000 31.90000),1950,15,7,15,7
2,2,195007,5,1800,195007,5,1800,NaN,10104927,PENNSYLVANIA,...,PUB,1950,10,3,POINT (-75.70000 40.58000),1950,10,3,10,4
3,3,195007,5,1830,195007,5,1830,NaN,10104928,PENNSYLVANIA,...,PUB,1950,10,1,POINT (-76.75000 40.60000),1950,10,1,None,None
4,4,195007,24,1440,195007,24,1440,NaN,10104929,PENNSYLVANIA,...,PUB,1950,10,2,POINT (-79.68000 41.63000),1950,1,2,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69199,39717,202006,4,159,202006,4,200,148826.0,896594,MISSOURI,...,CSV,2020,1,2,POINT (-94.33160 38.47190),2020,1,2,10,3
69200,39828,202006,28,1954,202006,28,1956,149511.0,901206,MINNESOTA,...,CSV,2020,10,1,POINT (-91.72380 44.03180),2020,10,1,14,1
69201,39961,202006,10,1718,202006,10,1724,147398.0,886388,OHIO,...,CSV,2020,10,1,POINT (-83.07910 39.37860),2020,10,1,10,3
69202,39964,202006,7,1741,202006,7,1742,147784.0,889521,FLORIDA,...,CSV,2020,14,5,POINT (-81.70400 28.79140),2020,1,5,1,3


In [12]:
landcover_mapper= {
1: "Trees",
2: "Tall Shrubs & Trees Mix (SEAK Only)",
3: "Shrubs & Trees Mix",
4: "Grass/Forb/Herb & Trees Mix",
5: "Barren & Trees Mix",
6: "Tall Shrubs (SEAK Only)",
7: "Shrubs",
8: "Grass/Forb/Herb & Shrubs Mix",
9: "Barren & Shrubs Mix",
10:"Grass/Forb/Herb",
11:"Barren & Grass/Forb/Herb Mix",
12:"Barren or Impervious",
13:"Snow or Ice",
14:"Water",
15:"Non-Processing Area Mask"
}

landuse_mapper = {
1: "Agriculture",
2: "Developed",
3: "Forest",
4: "Non-Forest Wetland",
5: "Other",
6: "Rangeland or Pasture",
7: "Non-Processing Area Mask",
}

In [45]:
gdf.drop('date', axis=1).to_file('tornado_loc.shp')

In [7]:
def compute_dist():

    
    dist_collections= {'dist':[],'coords':[]}
    for year in gdf.year_1.unique():
        if year<=1985:
            tar_year= 1985
        else: tar_year=year
        landuse= ee.Image('USFS/GTAC/LCMS/v2020-5/LCMS_CONUS_v2020-5_%d'%tar_year).select(2).eq(2)
        distance= landuse.fastDistanceTransform().sqrt().multiply(ee.Image.pixelArea().sqrt()).rename("distance")
        _gdf= gdf[gdf.year_1==year]
        points= ee.Geometry.MultiPoint([[_gdf.iloc[j].BEGIN_LON, _gdf.iloc[j].BEGIN_LAT] for j in range(len(_gdf))])
        return_dist= distance.sampleRegions(collection=points, scale=30, geometries=True)
        return_dict= return_dist.getInfo()
        dist= [float(feature['properties']['distance']/1000.) for feature in return_dict['features']]
        coords= [feature['geometry']['coordinates'] for feature in return_dict['features']]


        dist_collections['dist'].append(dist)
        dist_collections['coords'].append(coords)
        
        
    return dist_collections

In [8]:
dist= compute_dist()

In [9]:
dist_values= sum(dist['dist'], [])
dist_coords= sum(dist['coords'], [])

In [10]:
for i, [lon, lat] in enumerate(dist_coords):
    inds= gdf[(abs(gdf.BEGIN_LON-lon)<1e-3) & (abs(gdf.BEGIN_LAT-lat)<1e-3)].index
    gdf.loc[inds,'DistUrban']= dist_values[i]

## Retrieve CAPE from NLDAS-forcing

In [21]:
gdf.columns

Index(['Unnamed_ 0', 'BEGIN_YEAR', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMO',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE', 'CZ_TIMEZON', 'END_DATE_T', 'INJURIES_D',
       'INJURIES_I', 'DEATHS_DIR', 'DEATHS_IND', 'DAMAGE_PRO', 'DAMAGE_CRO',
       'SOURCE', 'MAGNITUDE', 'MAGNITUDE_', 'FLOOD_CAUS', 'CATEGORY',
       'TOR_F_SCAL', 'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_', 'TOR_OTHE_1',
       'TOR_OTHE_2', 'TOR_OTHE_3', 'BEGIN_RANG', 'BEGIN_AZIM', 'BEGIN_LOCA',
       'END_RANGE', 'END_AZIMUT', 'END_LOCATI', 'BEGIN_LAT', 'BEGIN_LON',
       'END_LAT', 'END_LON', 'EPISODE_NA', 'EVENT_NARR', 'DATA_SOURC',
       'year_1', 'LandCover', 'LandUse', 'year_2', 'LandCover_', 'LandUse_Be',
       'LandCove_1', 'LandUse_En', 'DistUrban', 'geometry'],
      dtype='object')

In [22]:
gdf.drop(['year_1', 'LandCover', 'LandUse', 'year_2'], axis=1, inplace=True)

In [27]:
gdf.rename({'LandCover_': 'BEGIN_LC',
            'LandUse_Be': 'BEGIN_LU',
            'LandCove_1': 'END_LC',
            'LandUse_En':'END_LU'}, axis=1, inplace=True)

In [6]:
gdf['date']= pd.to_datetime(gdf.BEGIN_YEAR.astype(str) + gdf.BEGIN_DAY.astype(str).str.zfill(2) + (gdf.BEGIN_TIME//100).astype(str).str.zfill(2), format='%Y%m%d%H')

In [7]:
gdf.date

0       1950-04-28 14:00:00
1       1950-04-29 15:00:00
2       1950-07-05 18:00:00
3       1950-07-05 18:00:00
4       1950-07-24 14:00:00
                ...        
69199   2020-06-04 01:00:00
69200   2020-06-28 19:00:00
69201   2020-06-10 17:00:00
69202   2020-06-07 17:00:00
69203   2020-06-02 14:00:00
Name: date, Length: 69204, dtype: datetime64[ns]

In [8]:

_gdf= gdf[gdf.date.dt.year>=1979]
cape_collections= {'cape':[],'coords':[], 'time': []}
for i, name, group in enumerate(_gdf.groupby('date')):
    if i%100==0: print('%d/%d'%(i, len(_gdf.date.unique())))
    img= ee.Image(name.strftime('NASA/NLDAS/FORA0125_H002/A%Y%m%d_%H00')).select('potential_energy') #A19800101_0000
    points= ee.Geometry.MultiPoint([[group.BEGIN_LON.iloc[i], group.BEGIN_LAT.iloc[i]] for i in range(len(group))])
    return_dist= img.sampleRegions(collection=points, scale=10000, geometries=True)
    return_dict= return_dist.getInfo()        
    cape= [float(feature['properties']['potential_energy']) for feature in return_dict['features']]
    coords= [feature['geometry']['coordinates'] for feature in return_dict['features']]    
    cape_collections['time'].append(name)
    cape_collections['coords'].append(coords)
    cape_collections['cape'].append(cape)


In [ ]:
cape_dict= get_CAPE()